In [33]:
import os
import json
from tqdm import tqdm
import random

In [31]:
exp_name = "covid19_rel_lianxiangjia"
data_home = "/home/wangyucheng/workplace/notebook/research/joint_extr/raw_data" + exp_name
data_output_dir = "/home/wangyucheng/workplace/notebook/research/joint_extr/ori_data" + exp_name
if not os.path.exists(data_output_dir):
    os.makedirs(data_output_dir)
    
train_label_path = os.path.join(data_home, "task2_train_label.json")
train_label = json.load(open(train_label_path, "r", encoding = "utf-8"))
train_text_path = os.path.join(data_home, "user_task2_train.json")
train_text = json.load(open(train_text_path, "r", encoding = "utf-8"))
test_text_path = os.path.join(data_home, "user_task2_valid.json")
test_text = json.load(open(test_text_path, "r", encoding = "utf-8"))

In [4]:
print(len(train_label), len(train_text), len(test_text))

1330 1330 570


In [17]:
idx2rel_list = {}
for sample in train_label:
    for k, v in sample.items():
        assert k not in idx2rel_list
        idx2rel_list[k] = v

idx2text = {}
for sample in train_text:
    for k, v in sample.items():
        assert k not in idx2text
        idx2text[k] = v

In [20]:
ori_train_data = []
for idx, text in idx2text.items():
    sample = {
        "id": idx,
        "text": text,
    }
    rel_list = []
    for rel in idx2rel_list[idx]:
        rel_list.append({
            "subject": rel[0],
            "predicate": rel[1],
            "object": rel[2],
        })
    sample["relation_list"] = rel_list
    ori_train_data.append(sample)

In [28]:
test_data = []
for sample in test_text:
    for k, v in sample.items():
        test_data.append({
            "id": k,
            "text": v,
        })

In [36]:
seed = 2333
k = 5
random.seed(seed)
random.shuffle(ori_train_data)
valid_num = int(len(ori_train_data) * (1 / k)) + 1

group_num = 0
for start_idx in tqdm(range(0, len(ori_train_data), valid_num)):
    end_idx = start_idx + valid_num
    valid_data = ori_train_data[start_idx:end_idx]
    train_data = ori_train_data[:start_idx] + ori_train_data[end_idx:]
#     print(len(train_data), len(valid_data))
    # ouput
    train_data_path = os.path.join(data_output_dir, "train_data_{}.json".format(group_num))
    valid_data_path = os.path.join(data_output_dir, "valid_data_{}.json".format(group_num))
    json.dump(train_data, open(train_data_path, "w", encoding = "utf-8"), ensure_ascii = False)
    json.dump(valid_data, open(valid_data_path, "w", encoding = "utf-8"), ensure_ascii = False)
    group_num += 1

 40%|████      | 2/5 [00:00<00:00, 12.00it/s]

1063 267
1063 267
1063 267


100%|██████████| 5/5 [00:00<00:00, 12.31it/s]

1063 267
1068 262


In [35]:
test_data_path = os.path.join(data_output_dir, "test_data.json")
json.dump(test_data, open(test_data_path, "w", encoding = "utf-8"), ensure_ascii = False)